In [1]:
import os
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

In [14]:
df = pd.read_csv("../static_merged_data.csv")
x = df.drop(labels="status", axis=1)
y = df[["status"]]
min_max_scale = MinMaxScaler()
x_scaled = min_max_scale.fit_transform(x)

def prepare_train_test_set(X, Y, num_classes):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, test_size=.2)
    train_labels = y_train.status.astype('category').cat.codes.to_numpy()
    test_labels = y_test.status.astype('category').cat.codes.to_numpy()
    X_train.to_numpy()
    X_test.to_numpy()
    return (X_train, X_test), (train_labels, test_labels)

In [15]:
(X_train, X_test), (train_labels, test_labels) = prepare_train_test_set(x,y,3)

In [130]:
from keras.models import Model
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Input, InputLayer
import tensorflow as tf
from keras import Sequential
from keras.layers import Activation 
import tensorflow_model_optimization as tfmot
model = tfmot.quantization.keras.quantize_annotate_model(Sequential([
    InputLayer(input_shape=(4,), name="input_layer"),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(64, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(128, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(256, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(128, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(16, activation="relu")),
    Dense(3, name="output_layer")
], name="best_quant_model"))

model.summary()

Model: "best_quant_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_annotate_46 (Quant  (None, 64)               320       
 izeAnnotate)                                                    
                                                                 
 quantize_annotate_47 (Quant  (None, 128)              8320      
 izeAnnotate)                                                    
                                                                 
 quantize_annotate_48 (Quant  (None, 256)              33024     
 izeAnnotate)                                                    
                                                                 
 quantize_annotate_49 (Quant  (None, 128)              32896     
 izeAnnotate)                                                    
                                                                 
 quantize_annotate_50 (Quant  (None, 16)          

In [16]:
from keras.models import load_model
import tensorflow_model_optimization as tfmot

with tfmot.quantization.keras.quantize_scope():
    model = load_model("./best_quant_model.h5")

In [131]:
from keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.optimizers import Adam



model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=(model.name + ".h5"),
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = model.fit(X_train, train_labels, epochs=120, validation_split=0.1, callbacks=[model_checkpoint_callback])

Epoch 1/120
2168/2168 [==============================] - 8s 3ms/step - loss: 0.7257 - accuracy: 0.6397 - val_loss: 0.2573 - val_accuracy: 0.8790
Epoch 2/120
2168/2168 [==============================] - 6s 3ms/step - loss: 0.2323 - accuracy: 0.9041 - val_loss: 0.2841 - val_accuracy: 0.8852
Epoch 3/120
2168/2168 [==============================] - 6s 3ms/step - loss: 0.2023 - accuracy: 0.9159 - val_loss: 0.1817 - val_accuracy: 0.9213
Epoch 4/120
2168/2168 [==============================] - 6s 3ms/step - loss: 0.1881 - accuracy: 0.9215 - val_loss: 0.4624 - val_accuracy: 0.8460
Epoch 5/120
2168/2168 [==============================] - 6s 3ms/step - loss: 0.1876 - accuracy: 0.9189 - val_loss: 0.1449 - val_accuracy: 0.9321
Epoch 6/120
2168/2168 [==============================] - 6s 3ms/step - loss: 0.1742 - accuracy: 0.9252 - val_loss: 0.2185 - val_accuracy: 0.8915
Epoch 7/120
2168/2168 [==============================] - 6s 3ms/step - loss: 0.1680 - accuracy: 0.9271 - val_loss: 0.2038 - val_ac

In [38]:
import tensorflow_model_optimization as tfmot
quantize_model = tfmot.quantization.keras.quantize_model
# q_aware stands for quantization aware
q_aware_model = quantize_model(model)

# Use `quantize_apply` to actually make the model quantization aware.
with tfmot.quantization.keras.quantize_scope():
    quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

# 'quantized_model' requires a recompile
quant_aware_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
#q_aware_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])



Model: "best_quant_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_11 (Quantize  (None, 4)                3         
 Layer)                                                          
                                                                 
 quant_dense_26 (QuantizeWra  (None, 64)               325       
 pperV2)                                                         
                                                                 
 quant_dense_27 (QuantizeWra  (None, 128)              8325      
 pperV2)                                                         
                                                                 
 quant_dense_28 (QuantizeWra  (None, 256)              33029     
 pperV2)                                                         
                                                                 
 quant_dense_29 (QuantizeWra  (None, 128)         

In [41]:
train_subset = X_train[0:20000]      # out of 60000
train_labels_subset = train_labels[0:20000]

quant_aware_model.fit(train_subset, train_labels_subset,batch_size=256,epochs=80, validation_split=0.1)

Epoch 1/80
71/71 [==============================] - 0s 6ms/step - loss: 0.2565 - accuracy: 0.8878 - val_loss: 0.2349 - val_accuracy: 0.8930
Epoch 2/80
71/71 [==============================] - 0s 6ms/step - loss: 0.2592 - accuracy: 0.8807 - val_loss: 0.2272 - val_accuracy: 0.9110
Epoch 3/80
71/71 [==============================] - 0s 6ms/step - loss: 0.2415 - accuracy: 0.8956 - val_loss: 0.2367 - val_accuracy: 0.8870
Epoch 4/80
71/71 [==============================] - 0s 6ms/step - loss: 0.3180 - accuracy: 0.8475 - val_loss: 0.2216 - val_accuracy: 0.9005
Epoch 5/80
71/71 [==============================] - 0s 6ms/step - loss: 0.3613 - accuracy: 0.8284 - val_loss: 0.3457 - val_accuracy: 0.8310
Epoch 6/80
71/71 [==============================] - 0s 6ms/step - loss: 0.3657 - accuracy: 0.8203 - val_loss: 0.4009 - val_accuracy: 0.7905
Epoch 7/80
71/71 [==============================] - 0s 6ms/step - loss: 0.3962 - accuracy: 0.8073 - val_loss: 0.3205 - val_accuracy: 0.8390
Epoch 8/80
71/71 [==

In [48]:
_, baseline_model_accuracy = model.evaluate(X_test,test_labels, verbose=0)
_, quant_aware_model_accuracy = quant_aware_model.evaluate(X_test,test_labels,verbose=0)
#_, q_aware_model_accuracy = q_aware_model.evaluate(X_test,test_labels,verbose=0)

with tfmot.quantization.keras.quantize_scope():
  loaded_model = tf.keras.models.load_model("newest_shit.tflite")

#_, newest_aware_model_accuracy = loaded_model.evaluate(X_test,test_labels,verbose=0)

print("Baseline test accuracy: ", baseline_model_accuracy)
print("Quantized test accuracy: ", quant_aware_model_accuracy)
#print("Full Quantized test accuracy: ", newest_aware_model_accuracy)

Baseline test accuracy:  0.9916969537734985
Quantized test accuracy:  0.9492994546890259


In [50]:
def representative_dataset():
    for sample in X_train.to_numpy()[:500,:]:
      yield [sample.astype(np.float32)]

In [57]:
# quantize with 8int weights and 16int activations
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]
tflite_16int_quant_model = converter.convert()



INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmp2ukwehxc\assets


INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmp2ukwehxc\assets


In [58]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
quantized_tflite_model = converter.convert()
quantized_tflite_file = "own_quantized_aware_model.tflite"

with open(quantized_tflite_file, "wb") as f:
    f.write(quantized_tflite_model)

INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmpy74rcv30\assets


INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmpy74rcv30\assets
c:\Users\sandr\source\repos\Monitoring_Converter_ML\venv\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [59]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model_int8 = converter.convert()

with open("full_int8_model.tflite", "wb") as f:
    f.write(tflite_quant_model_int8)

INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmpqvpwnovv\assets


INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmpqvpwnovv\assets
c:\Users\sandr\source\repos\Monitoring_Converter_ML\venv\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [60]:
import numpy as np

def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on over y image in the test dataset
    prediction_digits = []
    for i, test_image in enumerate(X_test.to_numpy()):
        if i % 1000 == 0:
            print("Evaluated on {n} results so far.".format(n=i))
        
        # Pre-processing: add batch dimension and convert to float32 to match with the model's input data format
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)

        # Run inference
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest probability
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)

    print("\n")

    # Compare prediction result with ground truth labels to calculate accuracy
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

In [61]:
interpreter = tf.lite.Interpreter(model_content=tflite_16int_quant_model)
interpreter.allocate_tensors()
interpreter_default = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter_default.allocate_tensors()

test_accuracy = evaluate_model(interpreter)
test_accuracy_default = evaluate_model(interpreter_default)

print("With 16int activations TFLite test_accuracy: ", test_accuracy)
print("Default TFLite test_accuracy: ", test_accuracy_default)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.
Evaluated on 10000 results so far.
Evaluated on 11000 results so far.
Evaluated on 12000 results so far.
Evaluated on 13000 results so far.
Evaluated on 14000 results so far.
Evaluated on 15000 results so far.
Evaluated on 16000 results so far.
Evaluated on 17000 results so far.
Evaluated on 18000 results so far.
Evaluated on 19000 results so far.


Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluate